In [1]:
## Details QA

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chains import create_tagging_chain_pydantic
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

import pandas as pd

In [3]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [4]:
conversation_history = ['Francis:  Hello, this is Francis from Francis Travel. Can i help you find a group tour today?',
 "User: Yes", 
                
               ]
# "Francis: Great! I'm here to help you find the perfect group tour. Can you please let me know what country you are looking to travel to?", 
# "User: India in April.",
# "Francis: That's great! India is a beautiful country to visit in April. How long do you want your trip to be?",
# "User: Why do you need to know?"

# , "Francis:  Where are you looking to travel to?", "User: India in April.",

In [5]:
import re
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [6]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key)

# stage analysis

In [7]:
from typing import Optional
from pydantic import BaseModel, Field

class TravelDetails(BaseModel):
    introduction: Optional[bool] = Field(
        False,
        description="Has francis introducted himself and asked if the user is interested in a group tour.",
    )
#     qualification: Optional[str] = Field(
#         ...,
#         description="Did the user confirm they are looking for a group tour or answer positivley when asked. If the user asks about a trip assume the answer is yes",
#         enum=["Yes", "No", "Unsure"]
#     )
    country: Optional[str] = Field(
        "",
        description="This is the name of the country the user is wanting to visit. If they name a place within a country always return the country",
        enum=["Cambodia", "Vietnam", "Thailand"]
    )
    departing_after: Optional[str] = Field(
        "",
        description="This is the first date from which the user can depart. If the user gives a month assume this is the first of the month. If not year if given return 2023. In the format '%d/%m/%Y'",
    )
    departing_before: Optional[str] = Field(
        "",
        description="This is the last date from which the user can depart. If the user gives a month assume this is the last day of the month. If not year if given return 2023. In the format '%d/%m/%Y'",
    )
    max_budget: Optional[int] = Field(
        0,
        description="This is the maximun amount of money the user is looking to spend on their trip.",
    )
    max_duration: Optional[int] = Field(
        None,
        description="This is the maximum duration of their trip."
    )
    min_duration: Optional[int] = Field(
        None,
        description="This is the minimum duration of their trip.",
    )

In [8]:
user_travel_details = TravelDetails(introduction=False,
                                # qualification="",
                                country="",
                                departing_after="",
                                departing_before="",
                                max_budget=None,
                                max_duration=None,
                                min_duration=None)

In [9]:
ask_for_dict = {"country":"what country are you looking to travel to?",
                "departing_after":"when are you looking to travel?",
                "departing_before":"when are you looking to travel?",
                "max_budget":"how much are you looking to spend?",
                "max_duration":"how long do you want your trip to be?",
                "min_duration":"how long do you want your trip to be?"
               }

In [10]:
def check_what_is_empty(user_travel_details):
    ask_for = []
    # Check if fields are empty
    for field, value in user_travel_details.dict().items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            ask_for.append(ask_for_dict[field])
    return ask_for

In [11]:
## checking the response and adding it
def add_non_empty_details(current_details: TravelDetails, new_details: TravelDetails):
    non_empty_details = {k: v for k, v in new_details.dict().items() if v not in [False, None, ""]}
    updated_details = current_details.copy(update=non_empty_details)
    return updated_details

In [12]:
# conversation_history = "\n".join(conversation_history)

# chain = create_tagging_chain_pydantic(TravelDetails, llm)
# res = chain.run(conversation_history)
# user_travel_details = add_non_empty_details(user_travel_details, res)
# ask_for = check_what_is_empty(user_travel_details)

In [13]:
# PROMPT_TEMPLATE = f"""You are currently in the trip detail phase of the conversation and are trying to get detail of the users trip to help find the the perfect trip. 
# You need to ask the user the following question to gather the required information:
# {ask_for[0]}"""

In [14]:
# if user_travel_details.dict()['qualification'] == 'no':
#     conversation_stage = "The user is not interested in a group tour so politely end the conversation. Ask them to come back if they ever are"
# elif user_travel_details.dict()['qualification'] == 'Unsure':
#     conversation_stage = "Explain what a group tour is. End by asking if they have any other questions?"
# else:
#     conversation_stage = PROMPT_TEMPLATE

In [15]:
def get_filtered_df(df, user_travel_details):
    trip_details_dict = user_travel_details.dict()
    filled_out_dictionary = {k: v for k, v in user_travel_details.dict().items() if v not in [False, None, "",0]}

    # convert dates to datetime format
    departing_after = pd.to_datetime(trip_details_dict["departing_after"], format='%d/%m/%Y')
    departing_before = pd.to_datetime(trip_details_dict["departing_before"], format='%d/%m/%Y')

    # Filtering the DataFrame
    filtered_df = df.copy()  # Make a copy of the original DataFrame to keep it intact
    
    # Iterate through the list of potential inputs
    for input_column in filled_out_dictionary.keys():
        if input_column == 'destination':
            filtered_df = filtered_df[filtered_df['countries'] == trip_details_dict["country"]]
        elif input_column == 'max_budget':
            filtered_df = filtered_df[filtered_df['cost'] <= trip_details_dict["max_budget"]]
        elif input_column == 'min_budget':
            filtered_df = filtered_df[filtered_df['cost'] >= trip_details_dict["min_budget"]]
        elif input_column == 'departing_after':
            filtered_df = filtered_df[filtered_df['start_date'] >= trip_details_dict["departing_after"]]
        elif input_column == 'departing_before':
            filtered_df = filtered_df[filtered_df['start_date'] <= trip_details_dict["departing_before"]]
        elif input_column == 'max_duration':
            filtered_df = filtered_df[filtered_df['duration'] <= trip_details_dict["max_duration"]]
        elif input_column == 'min_duration':
            filtered_df = filtered_df[filtered_df['duration'] >= trip_details_dict["min_duration"]]

    return filtered_df

In [16]:
class ItineraryDetails(BaseModel):
    itinerary_name: Optional[str] = Field(
        ...,
        description="The name of the itinerary the user has decided to take not the country",
    )
    itinerary_start_date: Optional[str] = Field(
        ...,
        description="The start date of the itinerary the user has decided to take. This is an exact date",
    )

In [17]:
user_itinerary_details = ItineraryDetails(itinerary_name="",
                                itinerary_start_date="")

In [18]:
found_itineraries = ["Best_of_Cambodia_Northern_Thailand","Cambodia_to_Vietnam_Night_Markets_Noodle-Making"]

In [19]:
available_dates = ["06/04/2023","13/04/2023","20/04/2023"]

In [20]:
URL = "www.affiliate_link_to_website.com"

In [21]:
asked_for = []

In [22]:
def check_conversation_stage(conversation_history, user_travel_details, user_itinerary_details):

    conversation_history = "\n".join(conversation_history)

    # extract travel details chain
    chain = create_tagging_chain_pydantic(TravelDetails, llm)
    res = chain.run(conversation_history)
    user_travel_details = add_non_empty_details(user_travel_details, res)
    ask_for = check_what_is_empty(user_travel_details)

    #load df
    df = pd.read_csv("test_travel_data.csv")
    
    # check number of unique itineraries
#     filtered_df = get_filtered_df(df, user_travel_details)
#     found_itineraries = filtered_df['itinerary_name'].unique()
    # available_dates = filtered_df['start_date'].unique()
    # URL = filtered_df['start_date'].unique()
    # print(available_dates)

    # if the number of itineraries is 0 give them alternatives e.g. different country/duration/time of year/budget
    # print(filtered_df['itinerary_name'].unique())

    if len(ask_for) == 0:
        # extract itinerary details chain
        chain = create_tagging_chain_pydantic(ItineraryDetails, llm)
        res = chain.run(conversation_history)
        user_itinerary_details = add_non_empty_details(user_itinerary_details, res)

    # # check if we have all validated details and if we have an itinerary from the user. 
    # # We then need present the list of dates and have the client decide
    if len(ask_for) == 0 and user_itinerary_details.itinerary_name in found_itineraries and user_itinerary_details.itinerary_start_date in available_dates:
        print('all details gathered, itinerary decided and date sorted')
        conversation_stage = f"""
        The user now has an itinerary and the dates they want to travel. 
        You are not booking the trip for the user, just share the companies url: {URL}"""
                                 
        return conversation_stage, user_travel_details


    # # check if we have all validated details and if we have an itinerary from the user. 
    # # We then need present the list of dates and have the client decide
    elif len(ask_for) == 0 and user_itinerary_details.itinerary_name in found_itineraries:
        dates_text = "\n".join(available_dates)
        conversation_stage = f"""
        The user has now decided on on the itinerary would like. We now need to check what dates they would like to travel on. The following dates are availab: 
        {dates_text}.
        Ask the user to pick a desired departure date."""
                                 
        return conversation_stage, user_travel_details

    # if we have all the validated details we need to present the list of itineraries and have the client decide
    elif len(ask_for) == 0 and len(found_itineraries) > 1:
        # all details gathered, check which itinerary the user wants
        print("all details gathered")
        conversation_stage = f"""Thank the user for providing the details. 
        Based on all the users needs, not just the answer to the last question, here is the list of itineraries that fit their needs: {found_itineraries}.
        Using the tools available provide a summeray of each itinerary.
        Ask the user to pick an itinerary so ."""
                                 
        return conversation_stage, user_travel_details


    # gather all the details we need from the client
    PROMPT_TEMPLATE = f"""You are currently in the trip detail phase of the conversation and are trying to get detail of the users trip to help find the the perfect trip. 
    If the user has just asked a follow up question in the conversation history, answer it.
    Once you have answered their question ALWAYS ask the user the following question to gather the required information.
    Follow up question:
    {ask_for[0]}"""
    
    # if user_travel_details.dict()['qualification'] == 'no':
    #     conversation_stage = "The user is not interested in a group tour so politely end the conversation. Ask them to come back if they ever are"
    # elif user_travel_details.dict()['qualification'] == 'Unsure':
    #     conversation_stage = "Explain what a group tour is. End by asking if they have any other questions?"
    # else:
    conversation_stage = PROMPT_TEMPLATE
    asked_for.append(ask_for[0])

    return conversation_stage, user_travel_details

In [23]:
# conversation_stage, user_travel_details = check_conversation_stage(conversation_history, user_travel_details)

In [24]:
# conversation_stage

In [25]:
# user_travel_details

# tools

In [26]:
# let's set up a dummy product catalog:
sample_product_catalog = """
Sleep Haven product 1: Luxury Cloud-Comfort Memory Foam Mattress
Experience the epitome of opulence with our Luxury Cloud-Comfort Memory Foam Mattress. Designed with an innovative, temperature-sensitive memory foam layer, this mattress embraces your body shape, offering personalized support and unparalleled comfort. The mattress is completed with a high-density foam base that ensures longevity, maintaining its form and resilience for years. With the incorporation of cooling gel-infused particles, it regulates your body temperature throughout the night, providing a perfect cool slumbering environment. The breathable, hypoallergenic cover, exquisitely embroidered with silver threads, not only adds a touch of elegance to your bedroom but also keeps allergens at bay. For a restful night and a refreshed morning, invest in the Luxury Cloud-Comfort Memory Foam Mattress.
Price: $999
Sizes available for this product: Twin, Queen, King

Sleep Haven product 2: Classic Harmony Spring Mattress
A perfect blend of traditional craftsmanship and modern comfort, the Classic Harmony Spring Mattress is designed to give you restful, uninterrupted sleep. It features a robust inner spring construction, complemented by layers of plush padding that offers the perfect balance of support and comfort. The quilted top layer is soft to the touch, adding an extra level of luxury to your sleeping experience. Reinforced edges prevent sagging, ensuring durability and a consistent sleeping surface, while the natural cotton cover wicks away moisture, keeping you dry and comfortable throughout the night. The Classic Harmony Spring Mattress is a timeless choice for those who appreciate the perfect fusion of support and plush comfort.
Price: $1,299
Sizes available for this product: Queen, King

Sleep Haven product 3: EcoGreen Hybrid Latex Mattress
The EcoGreen Hybrid Latex Mattress is a testament to sustainable luxury. Made from 100% natural latex harvested from eco-friendly plantations, this mattress offers a responsive, bouncy feel combined with the benefits of pressure relief. It is layered over a core of individually pocketed coils, ensuring minimal motion transfer, perfect for those sharing their bed. The mattress is wrapped in a certified organic cotton cover, offering a soft, breathable surface that enhances your comfort. Furthermore, the natural antimicrobial and hypoallergenic properties of latex make this mattress a great choice for allergy sufferers. Embrace a green lifestyle without compromising on comfort with the EcoGreen Hybrid Latex Mattress.
Price: $1,599
Sizes available for this product: Twin, Full

Sleep Haven product 4: Plush Serenity Bamboo Mattress
The Plush Serenity Bamboo Mattress takes the concept of sleep to new heights of comfort and environmental responsibility. The mattress features a layer of plush, adaptive foam that molds to your body's unique shape, providing tailored support for each sleeper. Underneath, a base of high-resilience support foam adds longevity and prevents sagging. The crowning glory of this mattress is its bamboo-infused top layer - this sustainable material is not only gentle on the planet, but also creates a remarkably soft, cool sleeping surface. Bamboo's natural breathability and moisture-wicking properties make it excellent for temperature regulation, helping to keep you cool and dry all night long. Encased in a silky, removable bamboo cover that's easy to clean and maintain, the Plush Serenity Bamboo Mattress offers a luxurious and eco-friendly sleeping experience.
Price: $2,599
Sizes available for this product: King
"""
with open("sample_product_catalog.txt", "w") as f:
    f.write(sample_product_catalog)

product_catalog = "sample_product_catalog.txt"

In [27]:
# Set up a knowledge base
def setup_knowledge_base(product_catalog: str = None):
    """
    We assume that the product knowledge base is simply a text file.
    """
    # load product catalog
    with open(product_catalog, "r") as f:
        product_catalog = f.read()

    text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
    texts = text_splitter.split_text(product_catalog)

    llm = OpenAI(temperature=0, openai_api_key=api_key)
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)
    docsearch = Chroma.from_texts(
        texts, embeddings, collection_name="product-knowledge-base"
    )

    knowledge_base = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
    )
    return knowledge_base


def get_tools(product_catalog):
    # query to get_tools can be used to be embedded and relevant tools found
    # see here: https://langchain-langchain.vercel.app/docs/use_cases/agents/custom_agent_with_plugin_retrieval#tool-retriever

    # we only use one tool for now, but this is highly extensible!
    knowledge_base = setup_knowledge_base(product_catalog)
    tools = [
        Tool(
            name="ProductSearch",
            func=knowledge_base.run,
            description="useful for when you need to answer questions about product information",
        )
    ]

    return tools

In [28]:
def get_tools():
    class DocumentInput(BaseModel):
        question: str = Field()


    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key)
    # Highlights_of_Morocco 
    tools = []
    files = [
        # https://abc.xyz/investor/static/pdf/2023Q1_alphabet_earnings_release.pdf
        {
            "name": "Best_of_Cambodia_Northern_Thailand",
            "path": "Best_of_Cambodia_Northern_Thailand.txt",
        },
        # https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q1-2023-Update
        {
            "name": "Cambodia_to_Vietnam_Night_Markets_Noodle-Making",
            "path": "Cambodia_to_Vietnam_Night_Markets_Noodle-Making.txt",
        },
        {
            "name": "Highlights_of_Morocco",
            "path": "Highlights_of_Morocco.txt",
        },
    ]

    for file in files:
        loader = TextLoader(file["path"])
        pages = loader.load_and_split()
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.split_documents(pages)
        embeddings = OpenAIEmbeddings(openai_api_key=api_key)
        retriever = FAISS.from_documents(docs, embeddings).as_retriever()

        # Wrap retrievers in a Tool
        tools.append(
            Tool(
                args_schema=DocumentInput,
                name=file["name"],
                description=f"useful when you want to answer questions about {file['name']} itinerary",
                func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
            )
        )
    return tools

# testing

In [29]:
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from langchain.agents import Tool
from langchain.schema.messages import SystemMessage

In [30]:
SALES_AGENT_TOOLS_PROMPT = """
Never forget your name is Francis. You work as a Travel Agent.
You work at company named Francis. Francis's business is the following: Francis is a context aware AI travel agent that works on finding users their dream group tour holiday.
You are contacting a potential prospect in order to find them a group toup holiday.
Your means of contacting the prospect is live chat.

Keep your responses in short length to retain the user's attention. Never produce lists, just answers.

Always think about the following conversation stage you are at before answering:

 - {conversation_stage}

You MUST respond according to the previous conversation history and the stage of the conversation you are at.
If you get asked about an itinerary use the tools available to you, do not make up an answer. If you do not know the answer tell the user.
Only generate one response at a time and act as Francis only!
Do not add Francis: to your output.

Previous conversation history:
{conversation_history}

Begin!
"""

                                 

In [31]:
def customize_prompt(conversation_history, conversation_stage, SALES_AGENT_TOOLS_PROMPT):

    conversation_history = "\n".join(conversation_history)
    
    
    from langchain import LLMChain, PromptTemplate
    prompt = PromptTemplate(
                template=SALES_AGENT_TOOLS_PROMPT,
                input_variables=[
                    "conversation_stage",
                    "conversation_history"
                ],
            )
    
    SALES_AGENT_TOOLS_PROMPT = prompt.format(conversation_stage=conversation_stage, 
                                     conversation_history=conversation_history)
    
    system_message = SystemMessage(
            content=(SALES_AGENT_TOOLS_PROMPT
            )
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
            system_message=system_message
    )

    return prompt
# print(prompt.messages[0].content)

In [32]:
# Define the tools
# tools = get_tools("sample_product_catalog.txt")
tools = get_tools()

In [33]:
# final_prompt = customize_prompt(conversation_history, conversation_stage, SALES_AGENT_TOOLS_PROMPT)
# # print(final_prompt.messages[0].content)

# conversation

In [50]:
user_travel_details = TravelDetails(introduction=False,
                                # qualification="",
                                country="",
                                departing_after="",
                                departing_before="",
                                max_budget=None,
                                max_duration=None,
                                min_duration=None)

In [51]:
user_itinerary_details = ItineraryDetails(itinerary_name="",
                                itinerary_start_date="")

In [52]:
conversation_history = ["Francis:  Hello, this is Francis from Francis Travel. Can i help you find a group tour today?"]

In [53]:
from langchain.agents import OpenAIMultiFunctionsAgent
def run_francis(input, conversation_history, user_travel_details, user_itinerary_details):
    user_input = f"User: {input}"

    conversation_history.append(user_input)

    conversation_stage, user_travel_details = check_conversation_stage(conversation_history, user_travel_details, user_itinerary_details)
    
    final_prompt = customize_prompt(conversation_history, conversation_stage, SALES_AGENT_TOOLS_PROMPT)
    
    # Create the agent
    # agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=final_prompt)
    agent = OpenAIMultiFunctionsAgent(llm=llm, tools=tools, prompt=final_prompt)
    
    # Run the agent with the actions
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False, max_iterations=5)
    #output = agent_executor.run("")
    
    francis = agent_executor.run(input)
    francis1 = f"Francis: {francis}"
    conversation_history.append(francis1)

    return francis, user_travel_details

In [54]:
human_input = "I want visit  Angkor Wat"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

That's a great choice! Angkor Wat is a magnificent temple complex in Cambodia. When are you looking to travel?


In [55]:
human_input = "April"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

Great! April is a wonderful time to visit Angkor Wat. How long are you planning to stay there?


In [57]:
human_input = "between 7 and 30 days"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

That's a flexible travel duration! Are you interested in exploring multiple destinations or just focusing on Angkor Wat?


In [58]:
human_input = "Angkor Wat and time through other south east asian countries"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

That sounds like a fantastic plan! Exploring Angkor Wat and other Southeast Asian countries will give you a diverse and enriching travel experience. To help me find the perfect trip for you, could you please let me know which specific countries you are interested in visiting?


In [60]:
human_input = "Cambodia, Vietnam, thailand"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

Great! Cambodia, Vietnam, and Thailand are all amazing destinations in Southeast Asia. To help me find the perfect trip for you, could you please let me know how much you are looking to spend?


In [61]:
user_travel_details

TravelDetails(introduction=True, country='Cambodia', departing_after='01/04/2023', departing_before='30/04/2023', max_budget=None, max_duration=30, min_duration=7)

In [62]:
human_input = "£10000"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

all details gathered
Thank you for providing your budget of £10000. Based on your budget and preferences, I have found two itineraries that may interest you:

1. Best of Cambodia & Northern Thailand: This itinerary includes visits to Angkor Wat in Cambodia and explores the highlights of Northern Thailand. You will have the opportunity to experience the rich history and culture of both countries.

2. Cambodia to Vietnam: Night Markets & Noodle-Making: This itinerary takes you from Cambodia to Vietnam, allowing you to explore the vibrant night markets and learn how to make traditional noodles. It's a great way to immerse yourself in the local culture.

Please let me know which itinerary you would like more information about or if you have any other questions.


In [65]:
human_input = "Could you give me some more details in the best of cambodia trip?"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

all details gathered
Certainly! Here is a more detailed breakdown of the Best of Cambodia & Northern Thailand itinerary:

Day 1: Arrive at any time.
Day 2: Start your adventure with a boat cruise through the villages, islands, and markets of the Mekong Delta. You'll also have the opportunity to visit traditional candy factories.
Day 3: Travel from Vietnam to Cambodia, crossing the border. Enjoy a cyclo tour through the city.
Day 4: Take a guided tour of the Tuol Sleng Museum (S-21 Prison) and Choeung Ek (Killing Fields). You can also choose to visit the Royal Palace, the National Museum, or the Central Market.
Day 5: Drive from Phnom Penh to Siem Reap, the home base for exploring the incredible Angkor Wat.
Day 6: Witness the stunning sunrise over the massive Angkor temple complex with a guided tour. You can also opt to visit the Tonlé Sap floating village and the Angkor National Museum. In the evening, enjoy a traditional meal of Khmer food at a G Adventures for Good-supported project.

In [66]:
human_input = "How much is it?"
francis_output, user_travel_details = run_francis(human_input, conversation_history, user_travel_details, user_itinerary_details)
print(francis_output)

all details gathered
The price for the Best of Cambodia & Northern Thailand itinerary starts at £1500 per person. Please note that prices may vary depending on the departure date and any additional options or upgrades you choose. Let me know if you have any other questions or if there's anything else I can assist you with!


In [64]:
for i in conversation_history:
    print(i)

Francis:  Hello, this is Francis from Francis Travel. Can i help you find a group tour today?
User: I want visit  Angkor Wat
Francis: That's a great choice! Angkor Wat is a magnificent temple complex in Cambodia. When are you looking to travel?
User: April
Francis: Great! April is a wonderful time to visit Angkor Wat. How long are you planning to stay there?
User: between 7 and 30 days
Francis: That's a flexible travel duration! Are you interested in exploring multiple destinations or just focusing on Angkor Wat?
User: Angkor Wat and time through other south east asian countries
Francis: That sounds like a fantastic plan! Exploring Angkor Wat and other Southeast Asian countries will give you a diverse and enriching travel experience. To help me find the perfect trip for you, could you please let me know which specific countries you are interested in visiting?
User: Cambodia, Vietnam, thailand
Francis: Great! Cambodia, Vietnam, and Thailand are all amazing destinations in Southeast Asia